<a href="https://colab.research.google.com/github/temilaj/BERT-playground/blob/master/BERT_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip \
    -O /tmp/bert-base.zip

--2021-02-13 05:11:14--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘/tmp/bert-base.zip’

/tmp/bert-base.zip  100%[===================>] 388.84M   151MB/s    in 2.6s    

2021-02-13 05:11:17 (151 MB/s) - ‘/tmp/bert-base.zip’ saved [407727028/407727028]



In [ ]:
!wget --no-check-certificate \
    https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz \
    -O /tmp/yelp_review_polarity_csv.tgz

--2021-02-13 05:13:52--  https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.214
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166373201 (159M) [application/x-tar]
Saving to: ‘/tmp/yelp_review_polarity_csv.tgz’

/tmp/yelp_review_po 100%[===================>] 158.67M  33.8MB/s    in 5.2s    

2021-02-13 05:13:58 (30.2 MB/s) - ‘/tmp/yelp_review_polarity_csv.tgz’ saved [166373201/166373201]



In [ ]:
import os
import zipfile

In [ ]:
local_zip = '/tmp/bert-base.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/bert-base')

zip_ref.close()

In [ ]:
import pandas as pd
# this is to extract the data from that .tgz file
import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# get all of the data out of that .tgz
yelp_reviews = tarfile.open('/tmp/yelp_review_polarity_csv.tgz')
yelp_reviews.extractall('/tmp/data')
yelp_reviews.close()


In [ ]:
# check out what the data looks like before you get started
# look at the training data set
train_df = pd.read_csv('/tmp/data/yelp_review_polarity_csv/train.csv', header=None)
print(train_df.head())

   0                                                  1
0  1  Unfortunately, the frustration of being Dr. Go...
1  2  Been going to Dr. Goldberg for over 10 years. ...
2  1  I don't know what Dr. Goldberg was like before...
3  1  I'm writing this review to give you a heads up...
4  2  All the food is great here. But the best thing...


In [ ]:
# look at the test data set
test_df = pd.read_csv('/tmp/data/yelp_review_polarity_csv/test.csv', header=None)
print(test_df.head())

   0                                                  1
0  2  Contrary to other reviews, I have zero complai...
1  1  Last summer I had an appointment to get new ti...
2  2  Friendly staff, same starbucks fair you get an...
3  1  The food is good. Unfortunately the service is...
4  2  Even when we didn't have a car Filene's Baseme...


In [ ]:
train_df[0] = (train_df[0] == 2).astype(int)
test_df[0] = (test_df[0] == 2).astype(int)

In [ ]:
print(train_df.head())
print(test_df.head())

   0                                                  1
0  0  Unfortunately, the frustration of being Dr. Go...
1  1  Been going to Dr. Goldberg for over 10 years. ...
2  0  I don't know what Dr. Goldberg was like before...
3  0  I'm writing this review to give you a heads up...
4  1  All the food is great here. But the best thing...
   0                                                  1
0  1  Contrary to other reviews, I have zero complai...
1  0  Last summer I had an appointment to get new ti...
2  1  Friendly staff, same starbucks fair you get an...
3  0  The food is good. Unfortunately the service is...
4  1  Even when we didn't have a car Filene's Baseme...


In [ ]:
bert_df = pd.DataFrame({
    'id': range(len(train_df)),
    'label': train_df[0],
    'alpha': ['q']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})

train_bert_df, dev_bert_df = train_test_split(bert_df, test_size=0.01)

In [ ]:
print(train_bert_df.head())

            id  label alpha                                               text
437800  437800      0     q  This place should be renamed \"cucumber heaven...
55982    55982      0     q  The food is decent enough but the service is q...
413770  413770      0     q  I don't think I've ever complained about price...
184931  184931      0     q  Long wait for service.  Less then mediocre foo...
482263  482263      1     q  Love this salon!  Dena and Chloe are great! I ...


In [ ]:
test_bert_df = pd.DataFrame({
    'id': range(len(test_df)),
    'text': test_df[1].replace(r'\n', ' ', regex=True)
})

In [ ]:
test_bert_df.head()

,id,text
0,0,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,3,The food is good. Unfortunately the service is...
4,4,Even when we didn't have a car Filene's Baseme...


In [ ]:
train_bert_df.to_csv('/tmp/data/train.tsv', sep='\t', index=False, header=False)
dev_bert_df.to_csv('/tmp/data/dev.tsv', sep='\t', index=False, header=False)
test_bert_df.to_csv('/tmp/data/test.tsv', sep='\t', index=False, header=False)